In [1]:
import numpy as np
import pandas as pd
# Evaluation libraries
from sklearn.metrics import accuracy_score, precision_score, f1_score ,recall_score
import tensorflow as tf
import keras
from keras.layers import Dense,Dropout
from keras.models import Sequential
from keras.callbacks import EarlyStopping
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder 
le = LabelEncoder() 
from keras.utils import np_utils
from sklearn.model_selection import KFold 
from sklearn.preprocessing import MinMaxScaler
# keras-tuner optimization
import kerastuner as kt
from kerastuner.tuners import RandomSearch
from numpy import average
from keras import backend as K
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.callbacks import EarlyStopping
# Visualization Data
import matplotlib.pyplot as plt

C:\Users\hager\AppData\Local\Temp/ipykernel_13024/2389151812.py:20: DeprecationWarning: `import kerastuner` is deprecated, please use `import keras_tuner`.
  import kerastuner as kt


In [2]:
list1=['radius_mean', 'perimeter_mean', 'area_mean', 'concavity_mean',
       'concave points_mean', 'radius_worst', 'texture_worst',
       'perimeter_worst', 'area_worst', 'smoothness_worst',
       'concave points_worst','label']

### Reading  training set and testing set

In [5]:
train= pd.read_csv('train_datasetset1.csv',encoding='utf-8')
train=train[list1]
y_train = train.label                           
X_train= train.drop('label',axis = 1 )
train.label.value_counts()

B    285
M    170
Name: label, dtype: int64

In [6]:
test= pd.read_csv('unseen_dataset1.csv',encoding='utf-8')
test=test[list1]
y_test = test.label                           
X_test= test.drop('label',axis = 1 )
test.label.value_counts()

B    72
M    42
Name: label, dtype: int64

In [7]:
num_features=len(X_train.columns)

### Sacling dataset

In [8]:
scaler = MinMaxScaler(feature_range=(0, 1))
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

### OneHotEncoder

In [9]:
encoder = LabelEncoder()
encoded_y_train= encoder.fit_transform(y_train)
y_train_categorical = np_utils.to_categorical(encoded_y_train)
y_test = encoder.transform(y_test)
y_test_categorical=np_utils.to_categorical(y_test)

In [10]:
Epochs=50
Batch_size=10

### Evaluation methods

In [11]:
from keras import backend as K

def recall_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

### KerasTuner 
#https://keras.io/keras_tuner/ 
KerasTuner is an easy-to-use, scalable hyperparameter optimization framework that solves the pain points of hyperparameter search. Easily configure your search space with a define-by-run syntax, then leverage one of the available search algorithms to find the best hyperparameter values for your models. KerasTuner comes with Bayesian Optimization, Hyperband, and Random Search algorithms built-in, and is also designed to be easy for researchers to extend in order to experiment with new search algorithms.
We 

In [12]:
def build_model(hp):
    num_units_min  =  50
    num_units_max  =  600
    num_units_step =  20

    dropout_min  =  .2
    dropout_max  =  0.9
    dropout_step =  0.1
    
    model = keras.Sequential()
    model.add(layers.Dense(units=hp.Int('unit1',  min_value=num_units_min,
                                                 max_value=num_units_max,
                                                 step=num_units_step),
                                                 input_dim=num_features, 
                                                kernel_initializer='he_uniform',
                           activation='relu'))
    model.add(layers.Dropout(hp.Float('dropout_1',min_value=dropout_min,
                                      max_value=dropout_max,
                                      step=dropout_step) ) )
    for i in range(hp.Int('num_layers', 1, 5)):
        model.add(layers.Dense(units=hp.Int('unitfor_'+ str(i),
                                            min_value=num_units_min,
                                            max_value=num_units_max,
                                            step=num_units_step),
                                           kernel_initializer='he_uniform',activation='relu'          
                                         ))
        model.add(layers.Dropout(hp.Float('dropout_2'+ str(i),min_value=dropout_min,
                                      max_value=dropout_max,
                                      step=dropout_step) ) )
    
    model.add(layers.Dense(2, activation='sigmoid'))
    model.compile(
        optimizer=keras.optimizers.Adam(hp.Choice('learning_rate',values=[ 1e-4])),
       loss='binary_crossentropy', 
       metrics=['acc',f1_m,precision_m, recall_m])
    return model


In [13]:
# Determine project name and the number of iteration 
tuner = RandomSearch(
    build_model,
    objective='val_acc',
    max_trials=2,
    project_name='RNN')


In [14]:

early_stopping = EarlyStopping(monitor='val_acc', patience=30, verbose=1)
callback_list = [ early_stopping ]
h=tuner.search(X_train, y_train_categorical,
             epochs=Epochs,
             batch_size=Batch_size, 
             callbacks=callback_list, validation_data=(X_test,y_test_categorical) )
            
model = tuner.get_best_models(num_models=1)[0]


Trial 2 Complete [00h 00m 13s]
val_acc: 0.9035087823867798

Best val_acc So Far: 0.9561403393745422
Total elapsed time: 00h 00m 28s
INFO:tensorflow:Oracle triggered exit


### get the best values of parameters

In [15]:
tuner.get_best_hyperparameters()[0].values

{'unit1': 350,
 'dropout_1': 0.30000000000000004,
 'num_layers': 2,
 'unitfor_0': 490,
 'dropout_20': 0.2,
 'learning_rate': 0.0001,
 'unitfor_1': 50,
 'dropout_21': 0.2}

In [16]:
AccuracyTrain=[]
PrecisionTrain=[]
RecallTrain=[]
F1Train=[]
Roc_auc_Train=[]
AccuracyTest=[]
PrecisionTest=[]
RecallTest=[]
F1Test=[]

for i in range(0,4):    
    if i==0:
        r=70
    elif i== 1:
        r=80
    elif i== 2:
        r=90
    elif i== 3:
        r=100
    elif i== 4:
        r=140
    
    kf = KFold(n_splits=10,random_state=r,shuffle=True) 
    for train_index, test_index in kf.split(X_train):
        X_t, X_val = X_train[train_index], X_train[test_index] 
        y_t, y_val = y_train_categorical[train_index], y_train_categorical[test_index]
        h=model.fit(X_t, y_t, epochs=Epochs, batch_size=Batch_size,  verbose=1, shuffle=True, validation_data=(X_val,y_val))

        AccuracyTrain.append(round(100*np.mean(h.history['val_acc']),2))
        
        PrecisionTrain.append(round(100*np.mean(h.history['val_precision_m']),2))
       
        RecallTrain.append(round(100*np.mean(h.history['val_recall_m']),2))
        
        F1Train.append(round(100*np.mean(h.history['val_f1_m']),2))
        

Epoch 1/50
41/41 [==============================] - 1s 12ms/step - loss: 0.1562 - acc: 0.9413 - f1_m: 0.9415 - precision_m: 0.9425 - recall_m: 0.9415 - val_loss: 0.1002 - val_acc: 0.9565 - val_f1_m: 0.9571 - val_precision_m: 0.9485 - val_recall_m: 0.9667
Epoch 2/50
41/41 [==============================] - 0s 5ms/step - loss: 0.1421 - acc: 0.9487 - f1_m: 0.9499 - precision_m: 0.9524 - recall_m: 0.9485 - val_loss: 0.0963 - val_acc: 1.0000 - val_f1_m: 1.0000 - val_precision_m: 1.0000 - val_recall_m: 1.0000
Epoch 3/50
41/41 [==============================] - 0s 5ms/step - loss: 0.1651 - acc: 0.9364 - f1_m: 0.9315 - precision_m: 0.9326 - recall_m: 0.9317 - val_loss: 0.0945 - val_acc: 1.0000 - val_f1_m: 1.0000 - val_precision_m: 1.0000 - val_recall_m: 1.0000
Epoch 4/50
41/41 [==============================] - 0s 5ms/step - loss: 0.1187 - acc: 0.9438 - f1_m: 0.9446 - precision_m: 0.9486 - recall_m: 0.9415 - val_loss: 0.0948 - val_acc: 1.0000 - val_f1_m: 1.0000 - val_precision_m: 1.0000 - val_

41/41 [==============================] - 0s 5ms/step - loss: 0.0781 - acc: 0.9756 - f1_m: 0.9767 - precision_m: 0.9760 - recall_m: 0.9780 - val_loss: 0.0530 - val_acc: 1.0000 - val_f1_m: 1.0000 - val_precision_m: 1.0000 - val_recall_m: 1.0000
Epoch 34/50
41/41 [==============================] - 0s 5ms/step - loss: 0.1024 - acc: 0.9682 - f1_m: 0.9648 - precision_m: 0.9618 - recall_m: 0.9683 - val_loss: 0.0522 - val_acc: 1.0000 - val_f1_m: 1.0000 - val_precision_m: 1.0000 - val_recall_m: 1.0000
Epoch 35/50
41/41 [==============================] - 0s 5ms/step - loss: 0.0961 - acc: 0.9560 - f1_m: 0.9599 - precision_m: 0.9572 - recall_m: 0.9631 - val_loss: 0.0600 - val_acc: 0.9783 - val_f1_m: 0.9800 - val_precision_m: 0.9800 - val_recall_m: 0.9800
Epoch 36/50
41/41 [==============================] - 0s 5ms/step - loss: 0.1032 - acc: 0.9609 - f1_m: 0.9594 - precision_m: 0.9634 - recall_m: 0.9561 - val_loss: 0.0541 - val_acc: 0.9783 - val_f1_m: 0.9800 - val_precision_m: 0.9800 - val_recall_m:

41/41 [==============================] - 0s 5ms/step - loss: 0.0842 - acc: 0.9707 - f1_m: 0.9671 - precision_m: 0.9662 - recall_m: 0.9683 - val_loss: 0.0183 - val_acc: 1.0000 - val_f1_m: 1.0000 - val_precision_m: 1.0000 - val_recall_m: 1.0000
Epoch 16/50
41/41 [==============================] - 0s 5ms/step - loss: 0.0773 - acc: 0.9731 - f1_m: 0.9779 - precision_m: 0.9787 - recall_m: 0.9780 - val_loss: 0.0149 - val_acc: 1.0000 - val_f1_m: 1.0000 - val_precision_m: 1.0000 - val_recall_m: 1.0000
Epoch 17/50
41/41 [==============================] - 0s 4ms/step - loss: 0.0829 - acc: 0.9658 - f1_m: 0.9660 - precision_m: 0.9639 - recall_m: 0.9683 - val_loss: 0.0140 - val_acc: 1.0000 - val_f1_m: 1.0000 - val_precision_m: 1.0000 - val_recall_m: 1.0000
Epoch 18/50
41/41 [==============================] - 0s 4ms/step - loss: 0.0674 - acc: 0.9780 - f1_m: 0.9768 - precision_m: 0.9783 - recall_m: 0.9756 - val_loss: 0.0147 - val_acc: 1.0000 - val_f1_m: 1.0000 - val_precision_m: 1.0000 - val_recall_m:

41/41 [==============================] - 0s 5ms/step - loss: 0.0781 - acc: 0.9731 - f1_m: 0.9742 - precision_m: 0.9756 - recall_m: 0.9732 - val_loss: 0.0123 - val_acc: 1.0000 - val_f1_m: 1.0000 - val_precision_m: 1.0000 - val_recall_m: 1.0000
Epoch 48/50
41/41 [==============================] - 0s 5ms/step - loss: 0.0790 - acc: 0.9707 - f1_m: 0.9681 - precision_m: 0.9711 - recall_m: 0.9659 - val_loss: 0.0119 - val_acc: 1.0000 - val_f1_m: 1.0000 - val_precision_m: 1.0000 - val_recall_m: 1.0000
Epoch 49/50
41/41 [==============================] - 0s 5ms/step - loss: 0.0806 - acc: 0.9731 - f1_m: 0.9705 - precision_m: 0.9729 - recall_m: 0.9683 - val_loss: 0.0123 - val_acc: 1.0000 - val_f1_m: 1.0000 - val_precision_m: 1.0000 - val_recall_m: 1.0000
Epoch 50/50
41/41 [==============================] - 0s 5ms/step - loss: 0.0811 - acc: 0.9804 - f1_m: 0.9794 - precision_m: 0.9785 - recall_m: 0.9805 - val_loss: 0.0134 - val_acc: 1.0000 - val_f1_m: 1.0000 - val_precision_m: 1.0000 - val_recall_m:

41/41 [==============================] - 0s 5ms/step - loss: 0.0649 - acc: 0.9780 - f1_m: 0.9766 - precision_m: 0.9778 - recall_m: 0.9756 - val_loss: 0.0175 - val_acc: 1.0000 - val_f1_m: 1.0000 - val_precision_m: 1.0000 - val_recall_m: 1.0000
Epoch 30/50
41/41 [==============================] - 0s 5ms/step - loss: 0.0633 - acc: 0.9682 - f1_m: 0.9694 - precision_m: 0.9662 - recall_m: 0.9729 - val_loss: 0.0115 - val_acc: 1.0000 - val_f1_m: 1.0000 - val_precision_m: 1.0000 - val_recall_m: 1.0000
Epoch 31/50
41/41 [==============================] - 0s 5ms/step - loss: 0.0671 - acc: 0.9707 - f1_m: 0.9733 - precision_m: 0.9736 - recall_m: 0.9732 - val_loss: 0.0142 - val_acc: 1.0000 - val_f1_m: 1.0000 - val_precision_m: 1.0000 - val_recall_m: 1.0000
Epoch 32/50
41/41 [==============================] - 0s 5ms/step - loss: 0.0609 - acc: 0.9731 - f1_m: 0.9729 - precision_m: 0.9729 - recall_m: 0.9729 - val_loss: 0.0161 - val_acc: 1.0000 - val_f1_m: 1.0000 - val_precision_m: 1.0000 - val_recall_m:

...

Epoch 7/50
41/41 [==============================] - 0s 6ms/step - loss: 0.0020 - acc: 1.0000 - f1_m: 1.0000 - precision_m: 1.0000 - recall_m: 1.0000 - val_loss: 1.2466e-05 - val_acc: 1.0000 - val_f1_m: 1.0000 - val_precision_m: 1.0000 - val_recall_m: 1.0000
Epoch 8/50
41/41 [==============================] - 0s 6ms/step - loss: 0.0035 - acc: 1.0000 - f1_m: 1.0000 - precision_m: 1.0000 - recall_m: 1.0000 - val_loss: 1.1942e-05 - val_acc: 1.0000 - val_f1_m: 1.0000 - val_precision_m: 1.0000 - val_recall_m: 1.0000
Epoch 9/50
41/41 [==============================] - 0s 6ms/step - loss: 0.0016 - acc: 1.0000 - f1_m: 1.0000 - precision_m: 1.0000 - recall_m: 1.0000 - val_loss: 1.0413e-05 - val_acc: 1.0000 - val_f1_m: 1.0000 - val_precision_m: 1.0000 - val_recall_m: 1.0000
Epoch 10/50
41/41 [==============================] - 0s 6ms/step - loss: 0.0024 - acc: 1.0000 - f1_m: 1.0000 - precision_m: 1.0000 - recall_m: 1.0000 - val_loss: 1.0986e-05 - val_acc: 1.0000 - val_f1_m: 1.0000 - val_precision_

Epoch 39/50
41/41 [==============================] - 0s 6ms/step - loss: 0.0020 - acc: 1.0000 - f1_m: 1.0000 - precision_m: 1.0000 - recall_m: 1.0000 - val_loss: 8.8369e-06 - val_acc: 1.0000 - val_f1_m: 1.0000 - val_precision_m: 1.0000 - val_recall_m: 1.0000
Epoch 40/50
41/41 [==============================] - 0s 6ms/step - loss: 0.0016 - acc: 1.0000 - f1_m: 1.0000 - precision_m: 1.0000 - recall_m: 1.0000 - val_loss: 7.0910e-06 - val_acc: 1.0000 - val_f1_m: 1.0000 - val_precision_m: 1.0000 - val_recall_m: 1.0000
Epoch 41/50
41/41 [==============================] - 0s 6ms/step - loss: 0.0017 - acc: 1.0000 - f1_m: 1.0000 - precision_m: 1.0000 - recall_m: 1.0000 - val_loss: 6.5159e-06 - val_acc: 1.0000 - val_f1_m: 1.0000 - val_precision_m: 1.0000 - val_recall_m: 1.0000
Epoch 42/50
41/41 [==============================] - 0s 6ms/step - loss: 0.0019 - acc: 1.0000 - f1_m: 1.0000 - precision_m: 1.0000 - recall_m: 1.0000 - val_loss: 4.5643e-06 - val_acc: 1.0000 - val_f1_m: 1.0000 - val_precisi

Epoch 21/50
41/41 [==============================] - 0s 6ms/step - loss: 0.0067 - acc: 0.9951 - f1_m: 0.9951 - precision_m: 0.9951 - recall_m: 0.9951 - val_loss: 1.3777e-05 - val_acc: 1.0000 - val_f1_m: 1.0000 - val_precision_m: 1.0000 - val_recall_m: 1.0000
Epoch 22/50
41/41 [==============================] - 0s 6ms/step - loss: 0.0014 - acc: 1.0000 - f1_m: 1.0000 - precision_m: 1.0000 - recall_m: 1.0000 - val_loss: 1.4000e-05 - val_acc: 1.0000 - val_f1_m: 1.0000 - val_precision_m: 1.0000 - val_recall_m: 1.0000
Epoch 23/50
41/41 [==============================] - 0s 6ms/step - loss: 0.0018 - acc: 1.0000 - f1_m: 1.0000 - precision_m: 1.0000 - recall_m: 1.0000 - val_loss: 1.4054e-05 - val_acc: 1.0000 - val_f1_m: 1.0000 - val_precision_m: 1.0000 - val_recall_m: 1.0000
Epoch 24/50
41/41 [==============================] - 0s 6ms/step - loss: 0.0015 - acc: 1.0000 - f1_m: 1.0000 - precision_m: 1.0000 - recall_m: 1.0000 - val_loss: 1.3907e-05 - val_acc: 1.0000 - val_f1_m: 1.0000 - val_precisi

Epoch 3/50
41/41 [==============================] - 0s 6ms/step - loss: 0.0015 - acc: 1.0000 - f1_m: 1.0000 - precision_m: 1.0000 - recall_m: 1.0000 - val_loss: 1.9987e-05 - val_acc: 1.0000 - val_f1_m: 1.0000 - val_precision_m: 1.0000 - val_recall_m: 1.0000
Epoch 4/50
41/41 [==============================] - 0s 6ms/step - loss: 0.0040 - acc: 0.9976 - f1_m: 0.9976 - precision_m: 0.9976 - recall_m: 0.9976 - val_loss: 1.1101e-05 - val_acc: 1.0000 - val_f1_m: 1.0000 - val_precision_m: 1.0000 - val_recall_m: 1.0000
Epoch 5/50
41/41 [==============================] - 0s 6ms/step - loss: 0.0015 - acc: 1.0000 - f1_m: 1.0000 - precision_m: 1.0000 - recall_m: 1.0000 - val_loss: 1.0773e-05 - val_acc: 1.0000 - val_f1_m: 1.0000 - val_precision_m: 1.0000 - val_recall_m: 1.0000
Epoch 6/50
41/41 [==============================] - 0s 6ms/step - loss: 0.0022 - acc: 1.0000 - f1_m: 1.0000 - precision_m: 1.0000 - recall_m: 1.0000 - val_loss: 1.3313e-05 - val_acc: 1.0000 - val_f1_m: 1.0000 - val_precision_m

Epoch 35/50
41/41 [==============================] - 0s 6ms/step - loss: 0.0015 - acc: 1.0000 - f1_m: 1.0000 - precision_m: 1.0000 - recall_m: 1.0000 - val_loss: 1.4408e-05 - val_acc: 1.0000 - val_f1_m: 1.0000 - val_precision_m: 1.0000 - val_recall_m: 1.0000
Epoch 36/50
41/41 [==============================] - 0s 6ms/step - loss: 0.0015 - acc: 1.0000 - f1_m: 1.0000 - precision_m: 1.0000 - recall_m: 1.0000 - val_loss: 1.3377e-05 - val_acc: 1.0000 - val_f1_m: 1.0000 - val_precision_m: 1.0000 - val_recall_m: 1.0000
Epoch 37/50
41/41 [==============================] - 0s 5ms/step - loss: 0.0014 - acc: 1.0000 - f1_m: 1.0000 - precision_m: 1.0000 - recall_m: 1.0000 - val_loss: 1.2815e-05 - val_acc: 1.0000 - val_f1_m: 1.0000 - val_precision_m: 1.0000 - val_recall_m: 1.0000
Epoch 38/50
41/41 [==============================] - 0s 5ms/step - loss: 0.0016 - acc: 1.0000 - f1_m: 1.0000 - precision_m: 1.0000 - recall_m: 1.0000 - val_loss: 1.4618e-05 - val_acc: 1.0000 - val_f1_m: 1.0000 - val_precisi

In [17]:
for i in range(0,4): 
    model.fit(X_train, y_train_categorical, epochs=Epochs, batch_size=Batch_size,  verbose=1, shuffle=True)
    y_p = model.predict(X_test)
    y_pred= np.argmax(y_p, axis=1)
    

    Accurcy_Test= accuracy_score(y_test,y_pred)
    Precision_Test=precision_score(y_test, y_pred,average='weighted')

    Recall_Test=recall_score(y_test, y_pred, average='weighted')
    F1_Test=f1_score(y_test, y_pred, average='weighted') 

    AccuracyTest.append(round(100*Accurcy_Test, 2))
    PrecisionTest.append(round(100*Precision_Test, 2))
    RecallTest.append(round(100*Recall_Test, 2))
    F1Test.append(round(100*F1_Test, 2))
    

Epoch 1/50
46/46 [==============================] - 0s 5ms/step - loss: 0.0027 - acc: 0.9978 - f1_m: 0.9978 - precision_m: 0.9978 - recall_m: 0.9978
Epoch 2/50
46/46 [==============================] - 0s 4ms/step - loss: 0.0079 - acc: 0.9956 - f1_m: 0.9957 - precision_m: 0.9957 - recall_m: 0.9957
Epoch 3/50
46/46 [==============================] - 0s 4ms/step - loss: 0.0013 - acc: 1.0000 - f1_m: 1.0000 - precision_m: 1.0000 - recall_m: 1.0000
Epoch 4/50
46/46 [==============================] - 0s 5ms/step - loss: 0.0013 - acc: 1.0000 - f1_m: 1.0000 - precision_m: 1.0000 - recall_m: 1.0000
Epoch 5/50
46/46 [==============================] - 0s 4ms/step - loss: 0.0063 - acc: 0.9978 - f1_m: 0.9978 - precision_m: 0.9978 - recall_m: 0.9978
Epoch 6/50
46/46 [==============================] - 0s 5ms/step - loss: 0.0027 - acc: 1.0000 - f1_m: 0.9989 - precision_m: 1.0000 - recall_m: 0.9978
Epoch 7/50
46/46 [==============================] - 0s 4ms/step - loss: 0.0017 - acc: 1.0000 - f1_m: 1.000

46/46 [==============================] - 0s 4ms/step - loss: 0.0012 - acc: 1.0000 - f1_m: 1.0000 - precision_m: 1.0000 - recall_m: 1.0000
Epoch 6/50
46/46 [==============================] - 0s 5ms/step - loss: 0.0012 - acc: 1.0000 - f1_m: 1.0000 - precision_m: 1.0000 - recall_m: 1.0000
Epoch 7/50
46/46 [==============================] - 0s 6ms/step - loss: 0.0013 - acc: 1.0000 - f1_m: 1.0000 - precision_m: 1.0000 - recall_m: 1.0000
Epoch 8/50
46/46 [==============================] - 0s 4ms/step - loss: 0.0014 - acc: 1.0000 - f1_m: 1.0000 - precision_m: 1.0000 - recall_m: 1.0000
Epoch 9/50
46/46 [==============================] - 0s 4ms/step - loss: 0.0157 - acc: 0.9956 - f1_m: 0.9957 - precision_m: 0.9957 - recall_m: 0.9957
Epoch 10/50
46/46 [==============================] - 0s 4ms/step - loss: 0.0019 - acc: 1.0000 - f1_m: 1.0000 - precision_m: 1.0000 - recall_m: 1.0000
Epoch 11/50
46/46 [==============================] - 0s 4ms/step - loss: 0.0026 - acc: 1.0000 - f1_m: 1.0000 - preci

46/46 [==============================] - 0s 4ms/step - loss: 0.0017 - acc: 1.0000 - f1_m: 1.0000 - precision_m: 1.0000 - recall_m: 1.0000
Epoch 10/50
46/46 [==============================] - 0s 5ms/step - loss: 0.0012 - acc: 1.0000 - f1_m: 1.0000 - precision_m: 1.0000 - recall_m: 1.0000
Epoch 11/50
46/46 [==============================] - 0s 4ms/step - loss: 0.0013 - acc: 1.0000 - f1_m: 1.0000 - precision_m: 1.0000 - recall_m: 1.0000
Epoch 12/50
46/46 [==============================] - 0s 4ms/step - loss: 0.0012 - acc: 1.0000 - f1_m: 1.0000 - precision_m: 1.0000 - recall_m: 1.0000
Epoch 13/50
46/46 [==============================] - 0s 4ms/step - loss: 0.0019 - acc: 1.0000 - f1_m: 1.0000 - precision_m: 1.0000 - recall_m: 1.0000
Epoch 14/50
46/46 [==============================] - 0s 4ms/step - loss: 0.0026 - acc: 1.0000 - f1_m: 1.0000 - precision_m: 1.0000 - recall_m: 1.0000
Epoch 15/50
46/46 [==============================] - 0s 4ms/step - loss: 0.0012 - acc: 1.0000 - f1_m: 1.0000 - p

46/46 [==============================] - 0s 4ms/step - loss: 0.0163 - acc: 0.9978 - f1_m: 0.9968 - precision_m: 0.9958 - recall_m: 0.9978
Epoch 14/50
46/46 [==============================] - 0s 4ms/step - loss: 0.0012 - acc: 1.0000 - f1_m: 1.0000 - precision_m: 1.0000 - recall_m: 1.0000
Epoch 15/50
46/46 [==============================] - 0s 4ms/step - loss: 0.0015 - acc: 1.0000 - f1_m: 1.0000 - precision_m: 1.0000 - recall_m: 1.0000
Epoch 16/50
46/46 [==============================] - 0s 4ms/step - loss: 0.0014 - acc: 1.0000 - f1_m: 1.0000 - precision_m: 1.0000 - recall_m: 1.0000
Epoch 17/50
46/46 [==============================] - 0s 4ms/step - loss: 0.0019 - acc: 1.0000 - f1_m: 1.0000 - precision_m: 1.0000 - recall_m: 1.0000
Epoch 18/50
46/46 [==============================] - 0s 4ms/step - loss: 0.0013 - acc: 1.0000 - f1_m: 1.0000 - precision_m: 1.0000 - recall_m: 1.0000
Epoch 19/50
46/46 [==============================] - 0s 4ms/step - loss: 0.0012 - acc: 1.0000 - f1_m: 1.0000 - p

In [18]:
finaltrain = pd.DataFrame([])

finaltrain = finaltrain.append({'AccuracyTrain' : round(np.mean(AccuracyTrain),2),
                                'PrecisionTrain':round(np.mean(PrecisionTrain),2),
                                'RecallTrain':round(np.mean(RecallTrain),2),  
                                'F1Train':round(np.mean(F1Train),2)} , ignore_index=True)

finaltrain.reindex(['AccuracyTrain','PrecisionTrain','RecallTrain','F1Train'], axis=1)

C:\Users\hager\AppData\Local\Temp/ipykernel_13024/3956621564.py:3: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  finaltrain = finaltrain.append({'AccuracyTrain' : round(np.mean(AccuracyTrain),2),


,AccuracyTrain,PrecisionTrain,RecallTrain,F1Train
0,99.66,99.63,99.64,99.64


In [19]:
ReultofTest=pd.DataFrame([])
ReultofTest=ReultofTest.append({'AccuracyTest' : round(np.mean(AccuracyTest),2),'PrecisionTest':round(np.mean(PrecisionTest),2),
             'RecallTest' : round(np.mean(RecallTest),2),'F1Test':round(np.mean(F1Test),2)}, ignore_index=True)

ReultofTest.reindex(['AccuracyTest','PrecisionTest','RecallTest','F1Test'], axis=1)

C:\Users\hager\AppData\Local\Temp/ipykernel_13024/1801123747.py:2: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  ReultofTest=ReultofTest.append({'AccuracyTest' : round(np.mean(AccuracyTest),2),'PrecisionTest':round(np.mean(PrecisionTest),2),


,AccuracyTest,PrecisionTest,RecallTest,F1Test
0,94.74,94.74,94.74,94.71
